# ***Building a Resume Parser Using NLP(Spacy) and Machine Learning***

In [ ]:
! pip install spacy_transformers
! pip install -U spacy
! pip install PyMuPDF

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
! git clone https://github.com/Shivam1431/Building_a_Resume_Parser_Using_NLP-Spacy_and_Machine-Learning.git

In [ ]:
cvData = json.load(open("/content/Building_a_Resume_Parser_Using_NLP-Spacy_and_Machine-Learning/data/training/train_data.json" , 'r'))

In [ ]:
len(cvData)

In [ ]:
!python -m spacy init fill-config /content/Building_a_Resume_Parser_Using_NLP-Spacy_and_Machine-Learning/data/training/base_config.cfg /content/Building_a_Resume_Parser_Using_NLP-Spacy_and_Machine-Learning/data/training/config.cfg

In [ ]:
cvData = cvData[0:2]

In [ ]:
def get_spacy_doc(file, data):
  nlp = spacy.blank('en')
  db = DocBin()

  for text , annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot["entities"]

    ents = []
    entity_indices = []

    for start , end , label in annot:
      skip_entity = False
      for idx in range(start,  end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue
        
        
      entity_indices = entity_indices + list(range(start , end))

      try :
        span = doc.char_span(start , end , label = label , alignment_mode = 'strict')

      except :
        continue

      if span is None:
        err_data = str([start , end]) + " " + str(text) + '\n'
        file.write(err_data)

      else :
        ents.append(span)

    try :
      doc.ents = ents
      db.add(doc)
    except :
      pass

  return db


In [ ]:
from sklearn.model_selection import train_test_split
train , test = train_test_split(cvData , test_size = 0.3)

In [ ]:
len(train) , len(test)

In [ ]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

In [ ]:
len(db.tokens)

In [ ]:
!python -m spacy train /content/Building_a_Resume_Parser_Using_NLP-Spacy_and_Machine-Learning/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

**MODEL TEST**

In [ ]:
nlp = spacy.load('/content/output/model-best')

In [ ]:
doc = nlp('My name is Shivam Mishra. I am Data Science intern at CodeClause. I have 4 years of experience in Teaching')
for ent in doc.ents:
  print(ent.text, " ->>>>> " , ent.label_)

In [ ]:
import sys, fitz

In [ ]:
fname = '/content/Building_a_Resume_Parser_Using_NLP-Spacy_and_Machine-Learning/data/test/Alice Clark CV.pdf'
doc = fitz.open(fname)

In [ ]:
doc_text = " "
for page in doc:
  doc_text = doc_text + str(page.get_text())

In [ ]:
doc_text = ''.join(doc_text.strip())

In [ ]:
doc_text

In [ ]:
doc = nlp(doc_text)
for ent in doc.ents:
  print(ent.text, " ->>>>> " , ent.label_)